In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [ ]:
df = pd.read_excel('catalog1.xlsx')

In [ ]:
df.head()

In [ ]:
df['Name'] = df['Категория'].str.lower().str.cat(df['Название'].str.lower(), sep=' ')
df.head()


In [ ]:
documents = df['Name']
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documents)


In [ ]:
print("Названия признаков (слов):")
print(vectorizer.get_feature_names_out())

In [ ]:
n_neighbors = 3
model = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine', algorithm='brute') 

In [ ]:
model.fit(tfidf_matrix)

In [ ]:
distances, indices = model.kneighbors(tfidf_matrix, n_neighbors=51)  # 51 сосед

In [ ]:
recommendations = {}
for i, neighbor_indices in enumerate(indices):
    # Фильтр: удалим сам товар (предполагаем, что он входит первым, либо найдём его индекс)
    filtered = [idx for idx in neighbor_indices if idx != i]
    # берем первые 50
    recommendations[df.iloc[i]['Артикул']] = df.iloc[filtered[:50]]['Артикул'].tolist()

In [ ]:
print(recommendations[tuple(recommendations.keys())[0]])
tuple(recommendations.keys())[0]

In [ ]:
new_df = pd.DataFrame({
    'A': [],
    'B': []
})
with open('catalog1_done.xlsx', 'a') as f:
    for i in range(len(recommendations.keys())):
        for j in range(len(recommendations[tuple(recommendations.keys())[i]])):
            new_row = f'{tuple(recommendations.keys())[i]}, {recommendations[tuple(recommendations.keys())[i]][j]}\n'
            f.write(new_row)